In [0]:
%run "./startup" 

In [0]:
params = WidgetParams({
    "sr_folio": str,    
    "sr_subproceso": str,
    "ind_vigencia": str,
    "config_indi": str
})
# Validar widgets
#params.validate()

In [0]:
conf = ConfManager()

#Archivos SQL
query = QueryManager()

#Conexion a base de datos
db = DBXConnectionManager()

In [0]:
#Query Extrae informacion 

statement = query.get_statement(
    "TRA_PATRIF_IND_VIV_0100_SOL_DESM_SELECT.sql",
    sr_folio=params.sr_folio,
    sr_subproceso=params.sr_subproceso,
    ind_vigencia=params.ind_vigencia,
    config_indi=params.config_indi
)

db.write_delta(f"DELTA_TRA_SOL_DESM_SELECT_{params.sr_folio}", db.read_data("default", statement), "overwrite")

if conf.debug:
    display(db.read_delta(f"DELTA_TRA_SOL_DESM_SELECT_{params.sr_folio}"))

In [0]:
9# Cargar tabla Delta en un DataFrame
df = spark.read.format("delta").table(f"DELTA_TRA_SOL_DESM_SELECT_{params.sr_folio}")

# Mostrar los primeros 100 registros
#df.show(100)

In [0]:
#INSERTA EN LA TABLA AUXILIAR

table_name = "CIERREN_DATAUX.TTCRXGRAL_MARCA_DESMARCA_INFO_AUX"

db.write_data(db.read_delta(f"DELTA_TRA_SOL_DESM_SELECT_{params.sr_folio}"), table_name, "default", "append")

In [0]:
#Se ejecuta el merge


statement = query.get_statement(
    "TRA_PATRIF_IND_VIV_0100_SOL_DESM_MERGE.sql",
    sr_folio=params.sr_folio,
)
execution = db.execute_oci_dml(
    statement=statement, async_mode=False
)

In [0]:
#Se elimina la Delta para liberar espacio
spark.sql(f"DROP TABLE IF EXISTS DELTA_TRA_SOL_DESM_SELECT_{params.sr_folio}")



#Se eliminan los registros de la tabla auxiliar
statement = query.get_statement(
    "TRA_PATRIF_IND_VIV_0100_SOL_DESM_DELETE_AUX.sql",
    sr_folio=params.sr_folio,
)
execution = db.execute_oci_dml(
    statement=statement, async_mode=False
)